In [216]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline


import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import open3d as o3d

import torch
import torch.nn as nn
import torch.nn.functional as F
if torch.cuda.is_available():
    print("Yeah we have a GPU!")
dev = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

from fastai.tabular.all import *

from gdeep.create_data.tori import make_torus_dataset, make_sphere_dataset
from gdeep.create_nets.utility import ToPyTorchNN
from gdeep.decision_boundary import GradientFlowDecisionBoundaryCalculator, StratificationGenerator


# Create Dataset and  Dataloaders

In [154]:
# Create Dataset
n_points = 1000

df = make_torus_dataset(entangled=True, n_points=n_points)
df_sphere = make_sphere_dataset(label=3, n_points=n_points, radius=6.0)
df = pd.concat((df, df_sphere), ignore_index=True)

px.scatter_3d(df, x='x1', y='x2', z='x3', color='label')

In [96]:
# Create Dataloader

# https://docs.fast.ai/tutorial.tabular.html

splits = RandomSplitter(valid_pct=0.2)(range_of(df))


to = TabularPandas(df, procs=[Normalize], cat_names=[], cont_names=[f'x{i+1}' for i in range(3)],
                   y_names='label', splits=splits, y_block = CategoryBlock)

dls = to.dataloaders(bs=128, device=dev)

In [97]:
dls.show_batch()

,x1,x2,x3,label
0,-2.435857,0.358161,-1.554776,0.0
1,3.799403,2.328331,-0.251293,1.0
2,2.524358,-1.858513,-5.116009,3.0
3,-1.504799,0.643737,-1.394506,0.0
4,4.631780,-0.378874,0.589307,1.0
5,-0.187125,-0.203301,-1.629924,0.0
6,4.137152,-0.925877,4.245789,3.0
7,3.301437,-4.316017,2.544112,3.0
8,-0.608491,-0.023174,1.491004,0.0
9,2.048242,0.681233,-1.269284,0.0


In [98]:
learn = tabular_learner(dls, layers=[16,32, 64, 64, 32, 3], metrics=accuracy)

learn.fit_one_cycle(100, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,1.108775,1.170315,0.345000,00:00
1,1.070152,1.081943,0.345000,00:00
2,1.028842,0.938774,0.345000,00:00
3,0.978435,0.810230,0.451667,00:00
4,0.911733,0.688468,0.595000,00:00
5,0.826488,0.546770,0.666667,00:00
6,0.735860,0.476325,0.666667,00:00
7,0.669471,0.458942,0.665000,00:00
8,0.600390,0.419550,0.666667,00:00
9,0.544205,0.343500,0.665000,00:00


In [199]:
model = ToPyTorchNN(learn.model)

class DBMulticlass(nn.Module):
    def __init__(self, strat_type: int=2):
        super().__init__()
        self.strat_type = strat_type

    def forward(self, x):
        y = torch.topk(x, self.strat_type).values
        return (y[:, 0] - y[:, self.strat_type - 1])**2 + 0.5

class LambdaLayer(nn.Module):
    def __init__(self, lam):
        super().__init__()
        self.lam = lam
    def forward(self, x):
        return self.lam(x)


strat_type = 2
model_prob = model_strat = nn.Sequential(
    model,
    LambdaLayer(lambda x: torch.softmax(x, dim=-1))
)

model_strat = nn.Sequential(
    model,
    LambdaLayer(lambda x: torch.softmax(x, dim=-1)),
    DBMulticlass(strat_type=strat_type)
)

In [194]:
n_steps = 1000
precision = 0.1

initial_points_batch = 4. * torch.rand((4000, 3)) - 2.
initial_points_batch.to(dev, non_blocking=True)
g = GradientFlowDecisionBoundaryCalculator(
            model=model_strat,
            initial_points=initial_points_batch,
            optimizer=lambda params: torch.optim.Adam(params)
)
g.step(n_steps)

db_sample = g.get_filtered_decision_boundary(precision).detach().numpy()

In [204]:
strat_gen = StratificationGenerator(model_prob, n_classes=3, df_points=db_df)

df_strat = strat_gen.create_normal_df(batch_size=128, verbose=True)

Starting computation of stratification type
batch number: 0  /  31
Starting computation of normal vectors
batch number: 0  /  31


In [260]:
for strat_type in itertools.combinations(range(3),2):
    strat_type = list(strat_type)
    df_strat_type = df_strat[df_strat['strat_type'].isin([strat_type])].values
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(df_strat_type[:,:3])
    pcd.normals = o3d.utility.Vector3dVector(df_strat_type[:,4:7])
    distances = pcd.compute_nearest_neighbor_distance()
    avg_dist = np.mean(distances)
    radius = 3 * avg_dist
    #poisson_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(pcd, depth=8, width=0, scale=1.1, linear_fit=False)[0]
    mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(pcd,o3d.utility.DoubleVector([radius, radius * 2]))
    mesh = mesh.simplify_quadric_decimation(100000)
    o3d.io.write_triangle_mesh(f"/home/reinauer/EPFL/giotto-deep/examples/MNIST_3D/Toy_Example_Tori_Sphere/mesh_{strat_type[0]}_{strat_type[1]}.ply", mesh)

In [261]:
strat_type = 3

model_strat = nn.Sequential(
    model,
    LambdaLayer(lambda x: torch.softmax(x, dim=-1)),
    DBMulticlass(strat_type=strat_type)
)

In [262]:
n_steps = 1000
precision = 0.1

initial_points_batch = 4. * torch.rand((4000, 3)) - 2.
initial_points_batch.to(dev, non_blocking=True)
g = GradientFlowDecisionBoundaryCalculator(
            model=model_strat,
            initial_points=initial_points_batch,
            optimizer=lambda params: torch.optim.Adam(params)
)
g.step(n_steps)

db_sample = g.get_filtered_decision_boundary(precision).detach().numpy()

In [264]:
db_sample.shape

(348, 3)

In [266]:
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(db_sample)

In [268]:
o3d.visualization.draw_geometries([pcd], point_show_normal=False)

In [270]:
o3d.io.write_point_cloud('/home/reinauer/EPFL/giotto-deep/examples/MNIST_3D/Toy_Example_Tori_Sphere/pcd_0_1_2.ply', pcd)

True